<a href="https://colab.research.google.com/github/Alishba18001778/Classification/blob/main/AE_MNIST_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import random
!pip install wget
from random import sample

from os import path
import h5py
import keras.backend as K
import wget

from keras.utils.np_utils import to_categorical
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D, UpSampling2D, BatchNormalization, AveragePooling2D, Dropout, Conv2DTranspose, SeparableConv2D, Add
from keras.models import Model
from keras import backend as K
from tensorflow.keras import datasets, layers, losses

In [25]:

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images = np.reshape(train_images, (len(train_images), 28, 28, 1))
test_images = np.reshape(test_images, (len(test_images), 28, 28, 1))

In [59]:

def my_model():
  input_img = Input(shape=(28, 28, 1))
  num_filter = 28
  kernel_size = 5
  strides = 1   
  
  x = Conv2D(num_filter, kernel_size, strides, activation='relu', padding='same', name = 'enc1')(input_img)

  x = Conv2D(1, kernel_size, strides, activation='relu', padding='same', name = 'enc2')(x) 

  x = AveragePooling2D((2, 2))(x)

  # x = Conv2D(1, kernel_size, strides, activation='relu', padding='same', name = 'enc4')(x) 
  # x = AveragePooling2D((2, 2))(x)   

  encoded = BatchNormalization()(x)
  y = Flatten()(encoded)
  y = Dense(128, activation='relu')(y)
  output = Dense(10, activation='softmax')(y)
  

  # x = UpSampling2D((2,1), interpolation = 'bilinear')(encoded)
  # x = Conv2DTranspose(1, kernel_size, strides, activation='relu', padding='same', name = 'dec1')(x)

  # # x = UpSampling2D((2,2), interpolation = 'bilinear')(x)        
  # # x = Conv2DTranspose(1, kernel_size, strides, activation='relu', padding='same', name = 'dec2')(x) 
  # x = BatchNormalization()(x)
  
  # x = Conv2DTranspose(num_filter, kernel_size, strides, activation='relu', padding='same', name = 'dec4')(x)
  # # x = BatchNormalization()(x) 
  # decoded = Conv2D(1,(1, 1), activation='sigmoid', name = 'output')(x)

  autoencoder = Model(input_img, output)
  autoencoder.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics = ['accuracy'])
  return autoencoder

autoencoder= my_model()

In [60]:
autoencoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
enc1 (Conv2D)                (None, 28, 28, 28)        728       
_________________________________________________________________
enc2 (Conv2D)                (None, 28, 28, 1)         701       
_________________________________________________________________
average_pooling2d_3 (Average (None, 14, 14, 1)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 1)         4         
_________________________________________________________________
flatten_3 (Flatten)          (None, 196)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               2521

In [61]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

checkpoint = ModelCheckpoint('models\\modelmnist-best.h5',
                                  verbose=0, monitor='val_loss',save_best_only=True, mode='auto')

autoencoder.fit(train_images, train_labels,
                epochs=200,
                batch_size=256,
                shuffle=True,
                validation_data=(test_images, test_labels),
                callbacks = [es, checkpoint]
                )

del autoencoder  # deletes the existing model
autoencoder = load_model('models\\modelmnist-best.h5')

Epoch 1/200
235/235 [==============================] - 3s 8ms/step - loss: 0.8483 - accuracy: 0.7261 - val_loss: 0.5938 - val_accuracy: 0.9395
Epoch 2/200
235/235 [==============================] - 1s 6ms/step - loss: 0.1522 - accuracy: 0.9564 - val_loss: 0.1261 - val_accuracy: 0.9693
Epoch 3/200
235/235 [==============================] - 1s 6ms/step - loss: 0.0952 - accuracy: 0.9715 - val_loss: 0.0884 - val_accuracy: 0.9735
Epoch 4/200
235/235 [==============================] - 1s 6ms/step - loss: 0.0711 - accuracy: 0.9791 - val_loss: 0.0679 - val_accuracy: 0.9775
Epoch 5/200
235/235 [==============================] - 1s 6ms/step - loss: 0.0531 - accuracy: 0.9845 - val_loss: 0.0592 - val_accuracy: 0.9811
Epoch 6/200
235/235 [==============================] - 1s 6ms/step - loss: 0.0444 - accuracy: 0.9871 - val_loss: 0.0571 - val_accuracy: 0.9804
Epoch 7/200
235/235 [==============================] - 1s 6ms/step - loss: 0.0359 - accuracy: 0.9892 - val_loss: 0.0627 - val_accuracy: 0.9794

In [6]:
test_images.shape
test_labels.shape

(10000,)

In [62]:
noise_factor_uniform = 0.4
noise_factor_normal = 0.25

x_test_noisy_uniform = test_images + noise_factor_uniform * np.random.uniform(size=test_images.shape) 
x_test_noisy_normal = test_images + noise_factor_normal * np.random.normal(size=test_images.shape)

x_test_noisy_uniform = np.clip(x_test_noisy_uniform, 0., 1.)
x_test_noisy_normal = np.clip(x_test_noisy_normal, 0., 1.)

In [63]:
predicted_uniform = autoencoder.evaluate(x_test_noisy_uniform,test_labels)
predicted_uniform = np.clip(predicted_uniform, 0., 1.)

predicted_normal = autoencoder.evaluate(x_test_noisy_normal,test_labels)
predicted_normal = np.clip(predicted_normal, 0., 1.)
print(predicted_uniform[1])

313/313 [==============================] - 1s 2ms/step - loss: 2.9426 - accuracy: 0.5695
0.5307999849319458


In [64]:

psnr_uniform_input = tf.image.psnr(x_test_noisy_uniform, test_images, max_val=1.0)

psnr_normal_input = tf.image.psnr(x_test_noisy_normal, test_images, max_val=1.0)

print('Uniform: ', np.mean(psnr_uniform_input), np.mean(predicted_uniform[1]) )
print('Gaussian: ', np.mean(psnr_normal_input), np.mean(predicted_normal[1]))


Uniform:  13.203412 0.5307999849319458
Gaussian:  14.857358 0.5695000290870667


In [65]:
def calculate_PSNR(test_images,noise_factor_uniform,noise_factor_normal,autoencoder):
  x_test_noisy_uniform = test_images + noise_factor_uniform * np.random.uniform(size=test_images.shape) 
  x_test_noisy_normal = test_images + noise_factor_normal * np.random.normal(size=test_images.shape)
  
  x_test_noisy_uniform = np.clip(x_test_noisy_uniform, 0., 1.)
  x_test_noisy_normal = np.clip(x_test_noisy_normal, 0., 1.)

  predicted_uniform = autoencoder.evaluate(x_test_noisy_uniform,test_labels)
  predicted_normal = autoencoder.evaluate(x_test_noisy_normal,test_labels)
  
  predicted_uniform = np.clip(predicted_uniform, 0., 1.)
  predicted_normal = np.clip(predicted_normal, 0., 1.)


  psnr_uniform_input = tf.image.psnr(x_test_noisy_uniform, test_images, max_val=1.0)

  psnr_normal_input = tf.image.psnr(x_test_noisy_normal, test_images, max_val=1.0)

  print('Uniform: ', np.mean(psnr_uniform_input), np.mean(predicted_uniform[1]))
  print('Gaussian: ', np.mean(psnr_normal_input), np.mean(predicted_normal[1]))

  return np.mean(psnr_uniform_input), np.mean(predicted_uniform[1])*100, np.mean(psnr_normal_input), np.mean(predicted_normal[1])*100

In [66]:

# print('----')
# noise_factor_uniform = 0.74
# noise_factor_normal = 0.5
# noise_factor_speckle = 2.8
# calculate_PSNR(test_images,noise_factor_uniform,noise_factor_normal,noise_factor_speckle,autoencoder)
PSNR_Results = []
for i in range(0,20):
  noise_factor_uniform = i*0.035
  noise_factor_normal = i*0.025
  
  PSNR_Results.append(calculate_PSNR(test_images,noise_factor_uniform,noise_factor_normal,autoencoder))
  print('----')

PSNR_Results = np.asarray(PSNR_Results)

313/313 [==============================] - 1s 2ms/step - loss: 0.0459 - accuracy: 0.9852
Uniform:  inf 0.9851999878883362
Gaussian:  inf 0.9851999878883362
----
313/313 [==============================] - 1s 2ms/step - loss: 0.0511 - accuracy: 0.9831
Uniform:  34.19517 0.9749000072479248
Gaussian:  34.578373 0.9830999970436096
----
313/313 [==============================] - 1s 2ms/step - loss: 0.0861 - accuracy: 0.9721
Uniform:  28.218037 0.8765000104904175
Gaussian:  28.617853 0.972100019454956
----
313/313 [==============================] - 1s 2ms/step - loss: 0.1828 - accuracy: 0.9428
Uniform:  24.718105 0.7554000020027161
Gaussian:  25.122688 0.942799985408783
----
313/313 [==============================] - 1s 2ms/step - loss: 0.3891 - accuracy: 0.8812
Uniform:  22.234531 0.70169997215271
Gaussian:  22.665247 0.8812000155448914
----
313/313 [==============================] - 1s 2ms/step - loss: 0.6879 - accuracy: 0.8152
Uniform:  20.311924 0.6690999865531921
Gaussian:  20.754143 0.8

In [58]:

from scipy.io import savemat

savemat('Noisy_Classification_MNIST_2x.mat',{'PSNR_Results':PSNR_Results})

from google.colab import files
files.download("Noisy_Classification_MNIST_2x.mat")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(AE-NCC for 2x compression)

[link text](https://github.com/Alishba18001778/Codes/blob/main/AE_MNIST_2x.png//)




AE-NCC for 4x compression
[link text](https://github.com/Alishba18001778/Codes/blob/main/AE_MNIST_4x.png//)